![tower bridge](london.jpg)

London, or as the Romans called it "Londonium"! Home to [over 8.5 million residents](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/populationandhouseholdestimatesenglandandwales/census2021unroundeddata#population-and-household-estimates-england-and-wales-data) who speak over [300 languages](https://web.archive.org/web/20080924084621/http://www.cilt.org.uk/faqs/langspoken.htm). While the City of London is a little over one square mile (hence its nickname "The Square Mile"), Greater London has grown to encompass 32 boroughs spanning a total area of 606 square miles! 

![underground train leaving a platform](tube.jpg)

Given the city's roads were originally designed for horse and cart, this area and population growth has required the development of an efficient public transport system! Since the year 2000, this has been through the local government body called **Transport for London**, or *TfL*, which is managed by the London Mayor's office. Their remit covers the London Underground, Overground, Docklands Light Railway (DLR), buses, trams, river services (clipper and [Emirates Airline cable car](https://en.wikipedia.org/wiki/London_cable_car)), roads, and even taxis.

The Mayor of London's office make their data available to the public [here](https://data.london.gov.uk/dataset). In this project, you will work with a slightly modified version of a dataset containing information about public transport journey volume by transport type. 

The data has been loaded into a **Snowflake** database called `TFL` with a single table called `JOURNEYS`, including the following data:

## TFL.JOURNEYS

| Column | Definition | Data type |
|--------|------------|-----------|
| `MONTH`| Month in number format, e.g., `1` equals January | `INTEGER` |
| `YEAR` | Year | `INTEGER` |
| `DAYS` | Number of days in the given month | `INTEGER` |
| `REPORT_DATE` | Date that the data was reported | `DATE` |
| `JOURNEY_TYPE` | Method of transport used | `VARCHAR` |
| `JOURNEYS_MILLIONS` | Millions of journeys, measured in decimals | `FLOAT` |

Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

You will execute SQL queries to answer three questions, as listed in the instructions.

In [33]:
-- most_popular_transport_types
SELECT J.JOURNEY_TYPE,
	SUM(JOURNEYS_MILLIONS) AS TOTAL_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS AS J
GROUP BY ALL
ORDER BY TOTAL_JOURNEYS_MILLIONS DESC


,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,Bus,24905.193947
1,Underground & DLR,15020.466544
2,Overground,1666.845666
3,TfL Rail,411.313421
4,Tram,314.689875
5,Emirates Airline,14.583718


In [34]:
-- emirates_airline_popularity
SELECT J.MONTH,
	J.YEAR,
	ROUND(J.JOURNEYS_MILLIONS,2) AS ROUNDED_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS AS J
WHERE J.JOURNEY_TYPE = 'Emirates Airline' AND J.JOURNEYS_MILLIONS IS NOT NULL
ORDER BY ROUNDED_JOURNEYS_MILLIONS DESC
LIMIT 5;

,MONTH,YEAR,ROUNDED_JOURNEYS_MILLIONS
0,5,2012,0.53
1,6,2012,0.38
2,4,2012,0.24
3,5,2013,0.19
4,5,2015,0.19


In [35]:
-- least_popular_years_tube
SELECT J.YEAR,
	J.JOURNEY_TYPE,
	SUM(J.JOURNEYS_MILLIONS) AS TOTAL_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS AS J
WHERE J.JOURNEY_TYPE LIKE '%Underground%'
GROUP BY ALL
ORDER BY TOTAL_JOURNEYS_MILLIONS
LIMIT 5;



,YEAR,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,2020,Underground & DLR,310.179316
1,2021,Underground & DLR,748.452544
2,2022,Underground & DLR,1064.859009
3,2010,Underground & DLR,1096.145588
4,2011,Underground & DLR,1156.647654


In [6]:
-- How many days on average did each reported month have, and what was the first and last month recorded by type of transportation?
SELECT J.JOURNEY_TYPE,
	ROUND(AVG(J.DAYS), 1) AS AVG_DAYS_PER_MONTH,
	MIN(J.REPORT_DATE) AS FIRST_REPORTED,
	MAX(J.REPORT_DATE) AS LAST_REPORTED
FROM TFL.JOURNEYS AS J
GROUP BY JOURNEY_TYPE
ORDER BY AVG_DAYS_PER_MONTH DESC;


,JOURNEY_TYPE,AVG_DAYS_PER_MONTH,FIRST_REPORTED,LAST_REPORTED
0,Underground & DLR,30.4,2010-06-30,2023-03-31
1,Overground,30.4,2010-06-30,2023-03-31
2,TfL Rail,30.4,2010-06-30,2023-03-31
3,Bus,30.4,2010-06-30,2023-03-31
4,Tram,30.4,2010-06-30,2023-03-31
5,Emirates Airline,30.4,2010-06-30,2023-03-31


In [10]:
-- Annual total by type of transport
WITH TOTAL_YEAR AS (
	SELECT YEAR,
		SUM(JOURNEYS_MILLIONS) AS TOTAL_YEARLY
	FROM TFL.JOURNEYS
	GROUP BY YEAR
	ORDER BY YEAR
)
	
SELECT *
FROM TOTAL_YEAR;


,YEAR,TOTAL_YEARLY
0,2010,3261.877309
1,2011,3429.603516
2,2012,3517.051285
3,2013,3580.343633
4,2014,3626.799056
5,2015,3752.506265
6,2016,3714.685291
7,2017,3678.435596
8,2018,3672.714708
9,2019,3633.525696


In [12]:
-- What was the average number of trips per year by transportation type, and how did the months within that year compare?

SELECT JOURNEY_TYPE,
	YEAR, 
	MONTH,
	JOURNEYS_MILLIONS,
	ROUND(AVG(JOURNEYS_MILLIONS) OVER (PARTITION BY JOURNEY_TYPE, YEAR), 2) AS AVG_YEARLY
FROM TFL.JOURNEYS
ORDER BY JOURNEY_TYPE, YEAR, MONTH



,JOURNEY_TYPE,YEAR,MONTH,JOURNEYS_MILLIONS,AVG_YEARLY
0,Bus,2010,1,189.114661,176.25
1,Bus,2010,2,181.604792,176.25
2,Bus,2010,3,175.883173,176.25
3,Bus,2010,4,183.428052,176.25
4,Bus,2010,5,160.422767,176.25
...,...,...,...,...,...
931,Underground & DLR,2022,8,95.314140,88.74
932,Underground & DLR,2022,9,102.666916,88.74
933,Underground & DLR,2022,10,74.262699,88.74
934,Underground & DLR,2022,11,92.618811,88.74
